<a href="https://colab.research.google.com/github/EmmanuelKnows/DS-Codveda/blob/main/Data_Cleaning_and__Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning and Preprocessing

## Install and import Necessary Libaries

Load dataset from url

In [18]:
# Import Libraries and Dependencies
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer

# Load dataset
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

print("Dataframe loaded successfully")

Dataframe loaded successfully


In [19]:
# print data frame
print(df.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [20]:
# Data Description

print(df.describe())

       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  


## Check for missing values

calculate percentage of missing value

In [21]:
# Check for missing values
print(df.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [22]:
# Calculate percentage of missing value
missing_percentage = (df.isnull().sum() / len(df)) * 100
print("Percentage of missing values per column:")
print(missing_percentage)

Percentage of missing values per column:
PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64


## Handling Missing Data
Missing data can be handled by removing (if the data is unsalvageable) or imputing (filling in with statistical averages).

**Removal**: We'll drop the Cabin column because over 70% of it is missing.

**Imputation**: We'll fill Age with the median and Embarked with the mode (most frequent).

In [23]:
# 1. Drop column with too many missing values
df.drop(columns='Cabin', inplace=True)

# 2. Impute Age with Median
df['Age'] = df['Age'].fillna(df['Age'].median())

# 3. Impute Embarked with Mode (most frequent value)
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

print("Missing values after cleaning:", df.isnull().sum())

Missing values after cleaning: PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


## Detecting and Removing Outliers
Outliers can skew our model's performance.

**Detecting**: From our data description, we can use the Interquartile Range (IQR) to detect which column has Outliers.

In [24]:
# Data Description

print(df.describe())

       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  891.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.361582    0.523008   
std     257.353842    0.486592    0.836071   13.019697    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   22.000000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   35.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  


**Removing**: Now We use the Interquartile Range (IQR) method to detect them in the Fare column and remove them.

In [25]:
# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = df['Fare'].quantile(0.25)
Q3 = df['Fare'].quantile(0.75)
IQR = Q3 - Q1

# Define bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove outliers
df = df[(df['Fare'] >= lower_bound) & (df['Fare'] <= upper_bound)]
print(f"Dataset size after outlier removal: {len(df)}")

Dataset size after outlier removal: 775


## Converting Categorical Variables

**Label Encoding**: For binary categories like Sex (Male/Female $\rightarrow$ 0/1).

**One-Hot Encoding**: For multi-category variables like Embarked (C, Q, S).

In [26]:
# Label Encoding for Sex
le = LabelEncoder()
df['Sex'] = le.fit_transform(df['Sex'])

print("Dataframe column sex after label encoding:")
print(df['Sex'].head(10))

Dataframe column sex after label encoding:
0     1
2     0
3     0
4     1
5     1
6     1
7     1
8     0
9     0
10    0
Name: Sex, dtype: int64


In [27]:
# One-Hot Encoding for Embarked
# This creates new columns: Embarked_C, Embarked_Q, Embarked_S
df = pd.get_dummies(df, columns=['Embarked'], prefix='Embarked')

print("Dataframe after One-Hot Encoding:")
print(df.head(10))

Dataframe after One-Hot Encoding:
    PassengerId  Survived  Pclass  \
0             1         0       3   
2             3         1       3   
3             4         1       1   
4             5         0       3   
5             6         0       3   
6             7         0       1   
7             8         0       3   
8             9         1       3   
9            10         1       2   
10           11         1       3   

                                                 Name  Sex   Age  SibSp  \
0                             Braund, Mr. Owen Harris    1  22.0      1   
2                              Heikkinen, Miss. Laina    0  26.0      0   
3        Futrelle, Mrs. Jacques Heath (Lily May Peel)    0  35.0      1   
4                            Allen, Mr. William Henry    1  35.0      0   
5                                    Moran, Mr. James    1  28.0      0   
6                             McCarthy, Mr. Timothy J    1  54.0      0   
7                      Palsson, M

## Normalization and Standardization

Numerical features often have different scales (e.g., Age is 0–80, Fare is 0–500). We need to bring them to a similar range.

In [28]:
scaler = StandardScaler()

# We scale 'Age' and 'Fare'
df[['Age', 'Fare']] = scaler.fit_transform(df[['Age', 'Fare']])

print(df.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
2            3         1       3   
3            4         1       1   
4            5         0       3   
5            6         0       3   

                                           Name  Sex       Age  SibSp  Parch  \
0                       Braund, Mr. Owen Harris    1 -0.528321      1      0   
2                        Heikkinen, Miss. Laina    0 -0.215182      0      0   
3  Futrelle, Mrs. Jacques Heath (Lily May Peel)    0  0.489381      1      0   
4                      Allen, Mr. William Henry    1  0.489381      0      0   
5                              Moran, Mr. James    1 -0.058613      0      0   

             Ticket      Fare  Embarked_C  Embarked_Q  Embarked_S  
0         A/5 21171 -0.779117       False       False        True  
2  STON/O2. 3101282 -0.729373       False       False        True  
3            113803  2.599828       False       False        True  
4            373450 -0.720161 